---
### **`Date: 2016/08/15`**
Experiment 3.0: Build program to fetch error code based on ticket history
---
title: experiment 3.0
author: Xiaoju (Ju) Zhang
**create: 2016/08/15**


### Working Directory
C:\cygwin64\home\zhangxx80\Work\Projects\errorCode\analysis\exp3.0

### Objective
The program is to create a database that contains history of error codes frequencies based on ticket receive date. With this database, it is possible to create a vector of error code frequency vector to find the similar instruments in long run.   

### Material and Methods

#### 1.  Build database of replicate counts
a. Download a whole year of result data from IDA (`result_cc_c_ReplicaidID_2015-07-13_2016-08-17.csv`) using jmp (`result_cc_c_replicateID_2015-07-13_2016-08-17.jsl`)
b. Since the dataset `result_cc_c_ReplicaidID_2015-07-13_2016-08-17.csv` is 44.4G, disolving the dataset into pieces based on instrument is necessary. Get list of all instruments and use python script to process the dataset. 

``` bash   
#Bash awk command to get the distinct instruments list
awk -F ',' '{ if (!x[$3]++) { print $3 } }' result_cc_c_ReplicaidID_2015-07-13_2016-08-17.csv > all_instruments.txt
```

#### 2. Parallel computing  
input: `messages_C16_2015-08-10_2015-09-10.csv`  

In [ ]:
#parallel computing.
import multiprocessing
import pandas as pd


dfMessage = pd.read_csv("./parallel/messages_C16_2015-08-10_2015-09-10.csv")
test_counts = {}

instruments = dfMessage['MODULESNDRM'].unique()

def test_counter(instrument):
    dfMessage_ = dfMessage[dfMessage['MODULESNDRM'] == instrument]
    test_count = dfMessage_.shape[0]
    test_counts[instrument] = test_count
    # print("{0}\t{1}".format(instrument, test_count))
    return test_counts

def mp_handler():
    results = {}
    p = multiprocessing.Pool()
    result_L = p.map(test_counter, instruments)
    for d in result_L:
        results.update(d)

    for k, v in results.iteritems():
        print k, v

if __name__ == '__main__':
    mp_handler()

output
```
C1692029 116
C1692020 115
C1692026 103
C1692024 171
C1600121 1814
C1600649 1063
C1600123 375
C1600122 451
C1600125 312
C1600127 1046
C1600640 300
...
```

#### 3. Timestamp comparison

In [4]:
import multiprocessing
import pandas as pd
import numpy as np

In [32]:
# dfMessage = pd.read_csv("./parallel/messages_2015-08-10_2015-09-10.csv", dtype={'MESSAGECODE':object})
ticket_fields = ['Serial Number', 'Receipt Date', 'Service Close Date', 'Completion Date', 'GSR Ticket Category']
dfTicket  = pd.read_csv("./parallel/ticket_search_2015-08-10_2015-09-10.csv", usecols=ticket_fields, parse_dates=['Receipt Date', 'Service Close Date', 'Completion Date'], 
    date_parser=lambda x: pd.to_datetime(x, format='%Y-%m-%d'))

In [33]:
dfTicket.head()

,Serial Number,Receipt Date,Service Close Date,Completion Date,GSR Ticket Category
0,C1600955,2015-09-10,2015-09-10,2015-12-18,Repair_CSC_PhoneFix
1,C400761,2015-09-10,2015-09-15,2015-09-25,Repair
2,C801212,2015-09-10,2015-09-10,2015-09-29,Repair
3,C400501,2015-09-10,2015-09-16,2015-11-18,Repair
4,C401377,2015-09-09,2015-10-01,2015-12-17,Repair_CSC_PhoneFix


In [30]:
dfMessage = pd.read_csv('./parallel/messages_2015-08-10_2015-09-10.csv', dtype={'MESSAGECODE':object}, parse_dates=['COMPLETIONDATE'], 
    date_parser=lambda x: pd.to_datetime(x, format='%m/%d/%Y %I:%M:%S %p'))

In [34]:
dfMessage.head()

,COMPLETIONDATE,MODULESNDRM,MESSAGECODE
0,2015-08-10 01:15:57,C1600802,9096
1,2015-08-10 01:15:57,C1600802,0711
2,2015-08-10 01:16:03,C1600802,0550
3,2015-08-10 01:24:16,C1600802,9096
4,2015-08-10 01:24:16,C1600802,3375


In [35]:
message_header = ['CompletionDate', 'SeriealNumber', 'ErrorCode']
dfMessage.columns = message_header

In [36]:
dfMessage.head()

,CompletionDate,SeriealNumber,ErrorCode
0,2015-08-10 01:15:57,C1600802,9096
1,2015-08-10 01:15:57,C1600802,0711
2,2015-08-10 01:16:03,C1600802,0550
3,2015-08-10 01:24:16,C1600802,9096
4,2015-08-10 01:24:16,C1600802,3375


In [23]:
ticket_header = ['SeriealNumber', 'ReceiptDate', 'ServiceCloseDate', 'CompletionDate', 'TicketCategory']
dfTicket.columns = ticket_header
dfTicket.head()

,SeriealNumber,ReceiptDate,ServiceCloseDate,CompletionDate,TicketCategory
0,C1600955,2015-09-10,2015-09-10,2015-12-18,Repair_CSC_PhoneFix
1,C400761,2015-09-10,2015-09-15,2015-09-25,Repair
2,C801212,2015-09-10,2015-09-10,2015-09-29,Repair
3,C400501,2015-09-10,2015-09-16,2015-11-18,Repair
4,C401377,2015-09-09,2015-10-01,2015-12-17,Repair_CSC_PhoneFix


In [54]:
dfMessage.iloc[0]['CompletionDate']

Timestamp('2015-08-10 01:15:57')

In [56]:
dfTicket.iloc[0]['Receipt Date']

Timestamp('2015-09-10 00:00:00')

In [57]:
dfMessage.iloc[0]['CompletionDate']

Timestamp('2015-08-10 01:15:57')

In [74]:
dfMessage.iloc[1]['CompletionDate']

Timestamp('2015-08-10 01:15:57')

In [77]:
#adding delta days
from datetime import timedelta
dfMessage.iloc[0]['CompletionDate'] + timedelta(days=25)

Timestamp('2015-09-04 01:15:57')

In [55]:
dfTicket.iloc[0]['Receipt Date'] > dfMessage.iloc[0]['CompletionDate']

True

#### 4. Fetch result table and message table for a particluar time window.

In [5]:
import pypyodbc

In [7]:
connection = pypyodbc.connect("Driver={SQL Server};Server=pabbto.oneabbott.com;Database=pabbto;uid=zhangxx80;pwd=Ej0_1a2I")

DatabaseError: (u'08001', u'[08001] [Microsoft][ODBC SQL Server Driver][DBNETLIB]SQL Server does not exist or access denied.')

In [17]:
import multiprocessing

def worker():
    """worker function"""
    print 'Worker'
    return


jobs = []
for i in range(5):
    p = multiprocessing.Process(target=worker)
    jobs.append(p)
    p.start()

In [ ]:
import multiprocessing
import time

data = (
    ['a', '2'], ['b', '4'], ['c', '6'], ['d', '8'],
    ['e', '1'], ['f', '3'], ['g', '5'], ['h', '7']
)

def mp_worker((inputs, the_time)):
    print " Processs %s\tWaiting %s seconds" % (inputs, the_time)
    time.sleep(int(the_time))
    print " Process %s\tDONE" % inputs

def mp_handler():
    p = multiprocessing.Pool(2)
    p.map(mp_worker, data)

mp_handler()